In [73]:
from  typing import Final
import numpy as np
from scipy.signal import get_window

In [74]:
verbose = False # To view output of arrays, set this to True, useful for understanding code

def verbose_print(message):
    if verbose:
        print(message)

The Leaky Integrate and Fire (LIF) neuronal model

Model Parameters:

In [75]:
TOTAL_SIMULATION_TIME: Final = 50  # Total simulation time in milliseconds [T]
SIMULATION_TIME_INTERVAL: Final = 0.1  # Simulation time step in milliseconds [dt]
STIMULUS_INITIALIZATION_TIME = 0  # Time at which stimulus begins in milliseconds [t_init]

RESTING_POTENTIAL_VOLTAGE: Final = -70  # Resting membrane potential in mV [V_rest]
MEMBRANE_RESISTANCE: Final = 1  # Membrane resistance in kOhm [Rm]
MEMBRANE_CAPACITANCE: Final = 5  # Capacitance of the membrane in uF [Cm]
REFRACTORY_PERIOD_TIME: Final = 1  # Refractory period duration in milliseconds [tau_ref]
SPIKE_THRESHOLD_VOLTAGE: Final = -40  # Voltage threshold for spike initiation in mV [V_th]
INPUT_CURRENT_AMPLITUDE: Final = 0.2  # Amplitude of input current in mA [I]
SPIKE_PEAK_VOLTAGE: Final = 50  # Voltage level representing a spike in mV [V_spike]

Simulation parameters:

In [76]:
# np.arange creates an array of time points for our simulation, specified by (start, stop, step).
# Starting at 0, we increment by the simulation time interval to simulate the neuron's behavior at each step.
# Stop parameter is defined as total simulation time + time interval. This ensures that the final simulation time is included in array. For example, if we did not add the time interval, and we had an interval of 3 and total time of 10, we would stop at time of 9 and never reach 10.

time_array = np.arange(0, TOTAL_SIMULATION_TIME + SIMULATION_TIME_INTERVAL,  SIMULATION_TIME_INTERVAL)
verbose_print(time_array)

In [77]:
# This initializes the membrane potential array (Vm) to the neuron's resting potential for all time steps.
# To achieve this, we first do np.ones(len(time_array)). We end up with a new array the length of time_array, except this time the entire array consists of 1. We then multiply this new array by the resting potential, making a new array where every value is just the resting potential.

# IMPORTANT: This array initialization serves as a baseline; actual potential values will be updated during the simulation part of the code, namely in the cell In 16, line 8.

# membrane potential is also referred as Vm -- to represent Voltage membrane
membrane_potential_array = np.ones(len(time_array)) * RESTING_POTENTIAL_VOLTAGE
verbose_print(membrane_potential_array)

In [78]:
# The membrane time constant [tau_m], refers to the speed at which a neuron's membrane potential returns to equilibrium (resting potential) after a stimuli. Thus, it also is how fast a neuron can respond to a new stimuli.
# Note that spikes are "all or nothing." They cannot be interrupted or prolonged during polarization/depolarization (firing states) or hyperpolarization (recharging state), hence why the value of [tau_m] is so important.

# To calculate:
# 1. Convert membrane resistance [Rm] from kiloohms to ohms by multiplying Rm by 1e3 (1000), as 1 kΩ equals 1000 Ω.
# 2. Convert membrane capacitance [Cm] from microfarads to farads by multiplying [Cm] by 1e-6, since 1 μF equals 1e-6 F.

# Membrane time constant [tau_m] in seconds
membrane_time_constant = MEMBRANE_RESISTANCE * 1e3 * MEMBRANE_CAPACITANCE * 1 * 1e-6
verbose_print(membrane_time_constant)

In [79]:
# We initialize an empty list, which we'll later use to record each time point which a 'spike' occurs.
# Note that a spike occurs when a neuron fires an action potential.
# In the context of the LIF model, a spike occurs when the membrane potential [Vm] reaches or surpasses the spike threshold voltage [V_th]. 

spikes = []

Defining the stimulus:

In [80]:
# The stimulus array models how many Amperes [amps] of electrical current  is provided as external stimuli to the neuron at a given time point
# The reason we're changing the stimulus is to measure the sensitivity of the neuron, and to see at what threshold an action potential is fired

# Notes:
# `INPUT_CURRENT_AMPLITUDE * 1e-3` converts the input current from milliamperes (mA) to amperes (A), since the INPUT_CURRENT_AMPLITUDE is given in mA and 1 mA = 1e-3 (-1000) amperes.

# In signal processing, a 'window' is a way of analyzing a specific segment of the signal instead of the entire thing.
# We form a window the shape of a triangle ('triang'). The length of this triangle is determined by len(time_array), which is the length of the time array.
# The reason we're using a triangle window is to apply a scalar to the amplitude (calculated by `INPUT_CURRENT_AMPLITUDE * 1e-3`). The amplitude is a scalar, but by multiplying it with the triangle window, we're changing the amplitude reaching the neuron at every time point.
# So with the triangular window, the amplitude reaching the neuron first increases linearly, until it reaches a peak. Afterward it starts decreasing linearly.

input_stimulus_array = (INPUT_CURRENT_AMPLITUDE * 1e-3 * get_window('triang', len(time_array)))
verbose_print(input_stimulus_array)

Simulation:

In [81]:
# Iterate over each time step (interval) in time array, excluding the last index, the notation that does this [:-1] is called negative array indexing in Python
# The reason we exclude the last index is to prevent the program from going out of bounds
for index, current_time in enumerate(time_array[:-1]):
    if current_time > STIMULUS_INITIALIZATION_TIME:
        # Calculate the membrane potential for the current time step based on the neuron's resting potential and the input stimulus
        current_membrane_potential_voltage = (RESTING_POTENTIAL_VOLTAGE * 1e-3) + (MEMBRANE_RESISTANCE * 1e-3) * input_stimulus_array[index]
        
        # '(membrane_potential_array[index] - current_membrane_potential_voltage)' is the potential difference of the membrane
        # 'np.exp(-SIMULATION_TIME_INTERVAL * -1e3 / membrane_time_constant)' we're converting the time interval to seconds and then divide it by the membrane time constant [tau_m]. Tau_m measures how long it takes for a membrane to reach 63% of its resting potential.
        membrane_potential_array[index + 1] = current_membrane_potential_voltage + (membrane_potential_array[index] - current_membrane_potential_voltage) * np.exp(-SIMULATION_TIME_INTERVAL * 1e-3 / membrane_time_constant)
        
        # Check for spike condition, aka action potential
        if membrane_potential_array[index] >= SPIKE_THRESHOLD_VOLTAGE * 1e-3:
            spikes.append(current_time) # record time of action potential
            membrane_potential_array[index] = SPIKE_PEAK_VOLTAGE * 1e-3 # set membrane potential equal to peak voltage to simulate spike
            # After spike, neuron enters hyperpolarization and is not able to spike for a while
            # Refactory Period Time + current time means neuron won't be able to spike until sometime in the future
            STIMULUS_INITIALIZATION_TIME = current_time + REFRACTORY_PERIOD_TIME * 1e-3